In [3]:
import arcpy
import pandas as pd
import os
import csv
import subprocess
from time import sleep


In [4]:

def set_years(cf):
    lu_tables = r"M:\code\landuse\lookup_tables"
    lc_dates = f"{lu_tables}/landcover_dates.csv" # T1 and T2 by cofips

    # read in years for county
    dates = pd.read_csv(lc_dates)
    dates = dates.set_index('co_fips')
    global T1_yr
    T1_yr = int(dates.loc[cf, 'T1'])
    global T2_yr
    T2_yr = int(dates.loc[cf, 'T2'])
    del dates
    return T1_yr, T2_yr

def MakeIterDir(parentDir, iterDir):
    """
    make a new folder in parentDir with the same name, indicated by iterDir, and the next iteration number
    """

    tnums = []
    for dir in os.listdir(parentDir):
        if iterDir in dir and len(dir) < 7:
            tnums.append(int(dir.replace(iterDir,"")))

    latestBN = f"{iterDir}{max(tnums)}"
    latestDir = os.path.join(parentDir,latestBN)
    latestDirNFiles = len([name for name in os.listdir(latestDir) if os.path.isfile(os.path.join(latestDir, name))])

    
    if latestDirNFiles == 0:
        # if the latest iteration directory is empty, set at iteration output directory
        LatestIterationDirectory = latestDir
        return LatestIterationDirectory
    else:
        #Otherwise create a new iteration directory
        NewIterDir = os.path.join(parentDir,f"{iterDir}{max(tnums)+1}")
        os.mkdir(NewIterDir)
        return NewIterDir

In [23]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
map = aprx.listMaps('Map1000ac')[0] #mapframe is Map1000ac
LO = aprx.listLayouts('Layout2')[0]
lyt = aprx.listLayouts('Layout2')[0]

water_poly = r"M:\projects\data_processing\data_processing.gdb/bay_and_ocean"
bay_poly = "M:\data\BayEstuary\Chesapeake_Bay_Alb.shp"
symb_lyr = r"M:\projects\tc_fact_sheet\symbology\hexsym_1ac_updated_colors_230130.lyrx"
bound_symb = r"M:\projects\tc_fact_sheet\symbology\1pt_hollow_70pctBLK_bound.lyrx"
boundPath = r"J:\GIS\CBP_Obj_1\data\county_sets\tl_2017_bay_county.shp"


#hide all layers
for lyr in map.listLayers():
    lyr.visible = False
    #
base_lyr = map.listLayers("Light Gray Base")[0]
base_lyr.visible = True

In [24]:
lcdates_csv = r"M:\projects\tc_fact_sheet\data\county_list_legendloc.csv"
codf = pd.read_csv(lcdates_csv)
#pandas upgrade required to use .query, PROPY env does not like that version 
omit = ['wash_11001', 'bedf_51515']
codf = codf[~codf['cofips'].isin(omit)]
len(codf)

204

In [26]:
#WITH SAFETIES

pngDIR = "M:/projects/tc_fact_sheet/map_exports/auto"
IterationDir = MakeIterDir(pngDIR, "test")

fail_list = []
counter = 0

for index, row in codf.iterrows():
    cf = row['cofips']
    T1 = row['T1']
    T1 = row['T2']
    LegendLoc = row['Legend']
    size = row['Size']

    FIPS = cf.split('_')[1]
    counter += 1
#     if counter > 15:
#         break
    print(f"{cf} {counter}/{len(codf)}", end='\r')

    FIPS = cf.split('_')[1]
    try:
        hexta = f"M:/projects/tc_fact_sheet/data/prod/{cf}/{cf}_hex_tajoined3.shp"
        map.addDataFromPath(hexta)
        hex_lyr = map.listLayers(f"{cf}_hex_tajoined3")[0] # FIND HEX LAYER IF IT EXISTS
        hex_lyr.visible = True
        desc = arcpy.Describe(hex_lyr)
        flds = desc.fields
        

        arcpy.SelectLayerByAttribute_management(hex_lyr, "CLEAR_SELECTION")
        hex_lyr.definitionQuery = "all_lu > ( 4046000/ 2) And wat_lu < (all_lu* 0.9)"
    except:
        msg = f"{cf} hex_lyr does not exist in {map.name}"
        print(msg)
        fail_list.append(msg)
        pass
    try:
        map.addDataFromPath(boundPath)
        bound_lyr = map.listLayers(f"tl_2017_bay_county")[0]
        arcpy.management.ApplySymbologyFromLayer(bound_lyr, bound_symb)
        bound_lyr.definitionQuery = f"GEOID = '{FIPS}'"
        
    except:
        msg2 = f"{cf} does not exist in bound_lyr"
        print(msg2)
        fail_list.append(msg2)
        hex_lyr.visible = False
        pass

    try:
        arcpy.management.ApplySymbologyFromLayer(hex_lyr, symb_lyr, "VALUE_FIELD tc_chg tc_chg", "MAINTAIN")
    except:
        print(cf)
        msg = f'{cf} hex_lyr failed symbolize'
        fail_list.append(msg)

    mf = lyt.listElements("MAPFRAME_ELEMENT")[0]
    # IF PICTURES ARE RE-ORDERED IN LAYOUT CONTENTS THIS WILL GET CONFUSED.

    LBM = lyt.listElements('PICTURE_ELEMENT')[0] # bottom middle
    LBL = lyt.listElements("PICTURE_ELEMENT")[1] # bottom left
    LBR = lyt.listElements("PICTURE_ELEMENT")[2] # bottom right
    mf.camera.setExtent(mf.getLayerExtent(bound_lyr, False, True))
    old_scale = mf.camera.scale
    if LegendLoc == 'r':
        LBM.visible = False
        LBR.visible = True
        LBL.visible = False
    elif LegendLoc == 'l':
        LBM.visible = False
        LBR.visible = False
        LBL.visible = True
    elif LegendLoc == 'b':
        LBM.visible = True
        LBR.visible = False
        LBL.visible = False
        mf.camera.scale = old_scale*1.2
        mf.camera.Y += -4000
        
    lyt.exportToPNG(f"{IterationDir}/{cf}_hexmap",resolution=300)

    map.removeLayer(hex_lyr)
    map.removeLayer(bound_lyr)

print('          ')
print(fail_list)
subprocess.Popen(f'explorer /select,{IterationDir}')
print(f"Complete! Outputs are located at... {IterationDir}")

lanc_51103 hex_lyr does not exist in Map1000ac
lanc_51103
           204/204
['lanc_51103 hex_lyr does not exist in Map1000ac', 'lanc_51103 hex_lyr failed symbolize']
Complete! Outputs are located at... M:/projects/tc_fact_sheet/map_exports/auto\test19


In [83]:
# for hl in map.listLayers(f"*_hex_tajoined"):
#     hl.visible = False

In [38]:


mf.camera.setExtent(mf.getLayerExtent(, False, True))


TypeError: M:/projects/tc_fact_sheet/data/prod/kent_10001/kent_10001_hex_tajoined2.shp

In [16]:
xcf = "alle_36003"
x_hexta = f"M:/projects/tc_fact_sheet/data/prod/{xcf}/{xcf}_hex_tajoined3.shp"
map.addDataFromPath(x_hexta)
x_lyr = map.listLayers(f"*_hex_tajoined3")[0]
x_lyr.name
x_lyr.definitionQuery = "all_lu > ( 4046000/ 2) And wat_lu < (all_lu* 0.9)"
mf.camera.setExtent(mf.getLayerExtent(x_lyr, False, True))

In [69]:
legend = lyt.listElements("MAPFRAME_ELEMENT")[0]

AttributeError: 'Extent' object has no attribute 'center'

In [21]:
mf.camera.setExtent(mf.getLayerExtent(x_lyr, False, True))

# rescale to make room for legend and move camera XY
old_scale = mf.camera.scale
mf.camera.scale = old_scale*1.2 

# mf.camera.X += 1000
mf.camera.Y += -4000

In [160]:
neg= map.listLayers(f"*neg_min_env")[0]


In [161]:
neg.definitionQuery = 

In [200]:
# IF PICTURES ARE RE-ORDERED IN LAYOUT CONTENTS THIS WILL GET CONFUSED.

LBM = lyt.listElements('PICTURE_ELEMENT')[0] # bottom middle
LBL = lyt.listElements("PICTURE_ELEMENT")[1] # bottom left
LBR = lyt.listElements("PICTURE_ELEMENT")[2] # bottom right

if LegendLoc == 'r':
    LBM.visible = False
    LBR.visible = True
    LBL.visible = False
elif LegendLoc == 'l':
    LBM.visible = False
    LBR.visible = False
    LBL.visible = True
elif LegendLoc == 'b':
    LBM.visible = True
    LBR.visible = False
    LBL.visible = False

In [ ]:
# Ripped this out now that there is full T2LU tabulations, can calculate on the fly
#         ALLWAT_exists = False #reset flag
#         for fld in flds:
#            if fld.name == 'ALLWAT':
#                 ALLWAT_exists = True
#         if not ALLWAT_exists:
#             arcpy.management.AddField(hex_lyr, "ALLWAT", "SHORT", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
#         arcpy.management.CalculateField(hex_lyr, "ALLWAT", "0", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
#         arcpy.management.SelectLayerByLocation(hex_lyr, "COMPLETELY_WITHIN", water_poly, "0 Meters", "NEW_SELECTION")
#         arcpy.management.CalculateField(hex_lyr, "ALLWAT", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
201   NaN
202   NaN
203   NaN
204   NaN
205   NaN
Name: test, Length: 204, dtype: float64

In [38]:
test = codf.groupby(['State']).agg({'T1':'mean'})
test

,T1
State,
DE,2013
MD,2013
NY,2013
PA,2013
VA,2014
WV,2014
